In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
API_KEY='a45d27d3-5b49-448d-9959-c3950744a918'
import requests
import io
import base64
import folium
import pydeck as pdk

In [4]:

user_input='https://www.zillow.com/gilbert-sc-29054/sold/?searchQueryState=%7B%22mapBounds%22%3A%7B%22north%22%3A34.095266698799236%2C%22east%22%3A-81.01262931542968%2C%22south%22%3A33.76539888865622%2C%22west%22%3A-81.5523326845703%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22lot%22%3A%7B%22min%22%3A43560%2C%22max%22%3A217800%7D%2C%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A70281%2C%22regionType%22%3A7%7D%2C%7B%22regionId%22%3A70295%2C%22regionType%22%3A7%7D%5D%2C%22pagination%22%3A%7B%7D%2C%22mapZoom%22%3A11%7D'
if user_input==None:
    st.write("Please provide a Zillow Listings Page link to proceed.")

if user_input is not None and user_input !="":
    
    
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"
    querystring = {
        "api_key": API_KEY,
        "url": user_input
    }

    listing_response = requests.request("GET", url, params=querystring)

    try:
        
        data = listing_response.json()['data']['cat1']['searchResults']['mapResults']
        df = pd.json_normalize(data)
        
        df['price']=df['hdpData.homeInfo.price']
        df['price']=df['price'].astype(str)
        df['price']=df['price'].str.replace(',','')
        df['price']=df['price'].astype(float)


        df['LOT_ACREAGE']=df['lotAreaString']
        df['LOT_ACREAGE']=df['LOT_ACREAGE'].str.strip()
        df['LOT_ACREAGE']=df['LOT_ACREAGE'].astype(str)
        df['LOT_ACREAGE']=df['LOT_ACREAGE'].str.replace('acres','')

        df['LOT_ACREAGE']=df['LOT_ACREAGE'].astype(float)

    except Exception as e:
        raise print(Exception)

In [6]:
df.columns

Index(['zpid', 'price', 'priceLabel', 'beds', 'baths', 'area', 'lotAreaString',
       'statusType', 'statusText', 'isFavorite', 'isUserClaimingOwner',
       'isUserConfirmedClaim', 'imgSrc', 'hasImage', 'visited', 'listingType',
       'shouldShowZestimateAsPrice', 'detailUrl', 'pgapt', 'sgapt',
       'has3DModel', 'hasVideo', 'isHomeRec', 'address',
       'hasAdditionalAttributions', 'isFeaturedListing', 'isShowcaseListing',
       'availabilityDate', 'latLong.latitude', 'latLong.longitude',
       'variableData.type', 'variableData.text', 'hdpData.homeInfo.zpid',
       'hdpData.homeInfo.streetAddress', 'hdpData.homeInfo.zipcode',
       'hdpData.homeInfo.city', 'hdpData.homeInfo.state',
       'hdpData.homeInfo.latitude', 'hdpData.homeInfo.longitude',
       'hdpData.homeInfo.price', 'hdpData.homeInfo.dateSold',
       'hdpData.homeInfo.bathrooms', 'hdpData.homeInfo.bedrooms',
       'hdpData.homeInfo.livingArea', 'hdpData.homeInfo.homeType',
       'hdpData.homeInfo.homeStatus'

In [8]:
df.to_csv('data.csv', index=False)